In [1]:
%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = 'svg'

## Import module

In [2]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import torch
import os
import numpy as np
from torchsummary import summary
from torchvision import transforms                                                                                                                                        
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 600
from PIL import Image
import pandas as pd

import sys
sys.path.append("../../../src/")
sys.path.append("../../")
import model
from datasets import imagenet
import config
from aux.utils import obtain_features_map, load_imgs, zscore, extract_valid
from aux.visualization import visualize_features_map
from utils.visualizations.visualize import concat_imgs, preprocess_arrays

import pytorch_ssim
import pandas as pd
import numpy as np

## Test function

#### mMetric_v1

In [102]:
def mMetric_v1(ori_activation_maps, opt_activation_maps, selected_filter,
               co1=1, co2=10000):
    """Metric, the first version.
    Args:
        ori_activation_maps: [batch_size, channels, height, width]
        opt_activation_maps: [batch_size, channels, height, width]
    """
    ori = ori_activation_maps.copy()
    opt = opt_activation_maps.copy()
    diff = ori_activation_maps-opt_activation_maps

    diff_sel = diff[:, selected_filter]
    ori_sel = ori[:, selected_filter]
    opt_sel = opt[:, selected_filter]
    opt_res = np.concatenate((opt[:, : selected_filter],
                              opt[:, selected_filter+1:]), axis=1)
    ori_res = np.concatenate((ori[:, : selected_filter],
                              ori[:, selected_filter+1:]), axis=1)

    (batch_size, channels, height, width) = ori.shape

    # SF term.
    SF_0 = np.sum(diff_sel == 0, axis=(1, 2))
    pixel_per_image = height * width
    SF_term = SF_0 / pixel_per_image
    print("=> SF_0: {}".format(SF_0))
    print("=> pixel_per_image: {}".format(pixel_per_image))
    print("=> SF_term: {}".format(SF_term))

    # RF term.
    ori_sel = np.expand_dims(ori_sel, axis=1)
#     print("ori_sel: ", ori_sel.shape)
    ori_sel = np.repeat(ori_sel, channels-1, axis=1)
#     print("ori_sel: ", ori_sel.shape)
    ori_sel_mask = ori_sel == 0
    opt_res = opt_res == 0
    opt_res = opt_res * ori_sel_mask
    pixel_per_image = np.sum(ori_sel_mask, axis=(1, 2, 3))
    RF_0_non_overlap = np.sum(opt_res == 1, axis=(1, 2, 3))
    RF_term = RF_0_non_overlap / pixel_per_image
    print("=> FF_0: {}".format(RF_0_non_overlap))
    print("=> pixel_per_image: {}".format(pixel_per_image))
    print("=> FF_term: {}".format(RF_term))

    over_metric = (co1 * SF_term + co2 * RF_term) / (co1 + co2)
    return over_metric

#### mMetirc_v2

In [81]:
def mMetric_v2(ori_activation_maps, opt_activation_maps, selected_filter,
               co1=1, co2=1):
    """Metric, the first version.
    Args:
        ori_activation_maps: [batch_size, channels, height, width]
        opt_activation_maps: [batch_size, channels, height, width]
    """
    ori = ori_activation_maps.copy()
    opt = opt_activation_maps.copy()
    diff = ori_activation_maps-opt_activation_maps

    diff_sel = diff[:, selected_filter]
    ori_sel = ori[:, selected_filter]
    opt_sel = opt[:, selected_filter]
    opt_res = np.concatenate((opt[:, : selected_filter],
                              opt[:, selected_filter+1:]), axis=1)
    ori_res = np.concatenate((ori[:, : selected_filter],
                              ori[:, selected_filter+1:]), axis=1)

    (batch_size, channels, height, width) = ori.shape
    pixel_per_image = height * width

    # SF term.
    
    diff_sel = np.reshape(diff_sel, (batch_size, -1))
    diff_sel = diff_sel - np.amax(diff_sel, axis=1, keepdims=True)
    diff_sel = np.exp(diff_sel)
    print(diff_sel)
    SF_0 = (diff_sel / diff_sel.sum(axis=1, keepdims=True)).mean(axis=1)
    SF_term = 1 - SF_0 / pixel_per_image
    print("=> SF_0: {}".format(SF_0))
    print("=> pixel_per_image: {}".format(pixel_per_image))
    print("=> SF_term: {}".format(SF_term))

    # RF term.
    ori_sel = np.expand_dims(ori_sel, axis=1)
    print("ori_sel: ", ori_sel.shape)
    ori_sel = np.repeat(ori_sel, channels-1, axis=1)
    print("ori_sel: ", ori_sel.shape)
    ori_sel_mask = ori_sel == 0
    opt_res = opt_res == 0
    opt_res = opt_res * ori_sel_mask
    pixel_per_image = np.sum(ori_sel_mask, axis=(1, 2, 3))
    RF_0_non_overlap = np.sum(opt_res == 1, axis=(1, 2, 3))
    RF_term = RF_0_non_overlap / pixel_per_image
    print("=> FF_0: {}".format(RF_0_non_overlap))
    print("=> pixel_per_image: {}".format(pixel_per_image))
    print("=> FF_term: {}".format(RF_term))

    over_metric = (co1 * SF_term + co2 * RF_term) / (co1 + co2)
    return over_metric

#### mMetric_v3

In [3]:
def mMetric_v3(ori_activation_maps, opt_activation_maps, selected_filter,
               co1=1, co2=1):
    ori = ori_activation_maps.copy()
    opt = opt_activation_maps.copy()
    diff = ori_activation_maps-opt_activation_maps

    diff_sel = diff[:, selected_filter]
    ori_sel = ori[:, selected_filter]
    opt_sel = opt[:, selected_filter]
    opt_res = np.concatenate((opt[:, : selected_filter],
                              opt[:, selected_filter+1:]), axis=1)
    ori_res = np.concatenate((ori[:, : selected_filter],
                              ori[:, selected_filter+1:]), axis=1)

    (batch_size, channels, height, width) = ori.shape

    ori_sel_ts = torch.from_numpy(ori_sel).unsqueeze(dim=1).cuda()
    opt_sel_ts = torch.from_numpy(opt_sel).unsqueeze(dim=1).cuda()
    
    ssim_sel = []
    rmses = []
    for index in range(30):
        ssim = pytorch_ssim.ssim(ori_sel_ts[index].unsqueeze(dim=0), 
                                 opt_sel_ts[index].unsqueeze(dim=0)).cpu().numpy()
        rmse = torch.sqrt(((ori_sel_ts[index].unsqueeze(dim=0)-
                         opt_sel_ts[index].unsqueeze(dim=0)) ** 2).mean()).cpu().numpy()
        ssim_sel.append(ssim)
        rmses.append(rmse)
        # ssim_loss = pytorch_ssim.SSIM(window_size=11)
        # print(ssim_loss(ori_sel, opt_sel))

    # RF
    ori_sel = np.expand_dims(ori_sel, axis=1)
    #print("ori_sel: ", ori_sel.shape)
    ori_sel = np.repeat(ori_sel, channels-1, axis=1)
    #print("ori_sel: ", ori_sel.shape)
    ori_sel_mask = ori_sel == 0
    opt_res_mask = opt_res != 0
    new_opt_res = opt_res * ori_sel_mask * opt_res_mask
    opt_res_ts = torch.from_numpy(new_opt_res).cuda()
    #print(opt_res_ts.shape)
    zero_mask = torch.zeros_like(opt_res_ts).cuda()
    
    ssim_res = []
    for index in range(30):
        ssim = pytorch_ssim.ssim(opt_res_ts[index].unsqueeze(dim=0), 
                                 zero_mask[index].unsqueeze(dim=0)).cpu().numpy()
        ssim_res.append(ssim)
    print("RMSE: {} - std {}".format(np.mean(rmses), np.std(rmses)))
    print("SSIM_SEL: {} - std {}".format(np.mean(ssim_sel), np.std(ssim_sel)))
    print("SSIM_RES: {} - std {}".format(np.mean(ssim_res), np.std(ssim_res)))
    lamba1 = 0.5
    lamba2 = 0.5
    ssim_sum = lamba1 * np.array(ssim_sel) + lamba2 * np.array(ssim_res)
    print("SSIM_SUM {} - std {}".format(np.mean(ssim_sum), np.std(ssim_sum)))

## Main

In [4]:
def main(exp, selected_layer, selected_filter, class_index=950):
    num_class = 30
    class_index = class_index
    print("=> class index: {}".format(class_index))
    backbone = "vgg16"
    resume = "037-0"
    color_map = "jet"
    model_dir = "/home/lincolnzjx/Desktop/Interpretation/saved/models"
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    
    ## Load modal from dict
    
    # model
    net = model.Network(backbone=backbone, num_classes=1000, 
                        selected_layer=selected_layer)
    net.to(device)
    
    # resume from model
    resume_exp = resume.split("-")[0]
    resume_epoch = resume.split("-")[1]
    print("Resume from model from exp: {} at epoch {}".format(resume_exp, resume_epoch))
    resume_path = os.path.join(model_dir, str(resume_exp), str(resume_epoch))
    ckpt = torch.load(resume_path, map_location=device)
    net.load_state_dict(ckpt, strict=False) 
    
    #summary(net, (3, 224, 224))
    
    ## Load Original Data 
    
    # Load data
    mean = [0.485, 0.456, 0.406]
    std = [0.229, 0.224, 0.225]
    reverse_mean = [-0.485, -0.456, -0.406]
    reverse_std = [1/0.229, 1/0.224, 1/0.225]
    train_transform = None
    train_transform = transforms.Compose([                                                                                                                                
        transforms.Resize((224, 224), interpolation=Image.BILINEAR),                                                                                              
        transforms.ToTensor(),                                                                                                                                            
        #transforms.Normalize(mean, std)                                                                                                                                   
           ])         
    trainset = imagenet.ImageNet(root="/media/lincolnzjx/HardDisk/Datasets/", 
                                 is_train=True, transform=train_transform)
    
    trainset.set_data([class_index], num_class)
    imgs_path = []                                                                                                                                                            
    images = []
    labels = []
    for img, label, img_path in trainset:                                                                                                                                     
        images.append(img.unsqueeze(0))                                                                                                                                       
        labels.append(label)                                                                                                                                                  
        imgs_path.append(img_path)  
    
    ## Create some need path
    
    save_dir = "../../../saved/pack/"
    ################### Hyper-Parameter #######################
    # exp = resume_exp
    # epoch = resume_epoch
    ##########################################################
    ab_path = os.path.join(save_dir, exp)
    
    index2image = {index: item.split("/")[-1].split(".")[0] for index, item in enumerate(imgs_path)}
    index2image
    
    images_cpu = np.array([image.detach().clone().cpu().numpy().squeeze() for image in images])
    
    # Load image
    # ext may be different.
    optimized_data, valid_imgs_path, valid_imgs_index = load_imgs(ab_path, imgs_path, non_exists_ok=True, ext=".png")
    valid_imgs, valid_labels = extract_valid(images, labels, valid_imgs_index)
    optimized_data_zscore = zscore(optimized_data, mean, std)
    images_zscore = zscore(images_cpu, mean, std)
    
    # Move to device
    opt_image = torch.from_numpy(optimized_data_zscore).to(device)
    original_image = torch.from_numpy(images_zscore).to(device)
    
    ## Obtain feature from conv layers.
    
    ori_activation_maps = net.get_activation_maps(original_image, 
                                                  selected_layer=selected_layer)[0]
    opt_activation_maps = net.get_activation_maps(opt_image, 
                                                  selected_layer=selected_layer)[0]
    return ori_activation_maps, opt_activation_maps
    

#### L1F16

In [6]:
exp = "052490"
selected_layer = 1
selected_filter = 16
class_index = 950
ori_activation_maps, opt_activation_maps = main(exp, selected_layer, 
                                                selected_filter, class_index)

=> class index: 950
Only Keep 1th layers before.
Resume from model from exp: 037 at epoch 0
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 224, 224]           1,792
              ReLU-2         [-1, 64, 224, 224]               0
             VGG16-3         [-1, 64, 224, 224]               0
Total params: 1,792
Trainable params: 1,792
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.57
Forward/backward pass size (MB): 73.50
Params size (MB): 0.01
Estimated Total Size (MB): 74.08
----------------------------------------------------------------
Len of new dataset is :30
../../../saved/pack/052490
Load from ../../../saved/pack/052490/46785/n07747607_10061.png
Load from ../../../saved/pack/052490/46785/n07747607_10068.png
Load from ../../../saved/pack/052490/46785/n07747607_10110.png
Load from ../../../saved/pack/

##### mMetric_v1

In [103]:
over_metric = mMetric_v1(ori_activation_maps, ori_activation_maps, selected_filter=selected_filter)
print(over_metric)
print(np.mean(over_metric))

=> SF_0: [50176 50176 50176 50176 50176 50176 50176 50176 50176 50176 50176 50176
 50176 50176 50176 50176 50176 50176 50176 50176 50176 50176 50176 50176
 50176 50176 50176 50176 50176 50176]
=> pixel_per_image: 50176
=> SF_term: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1.]
=> FF_0: [453395 109037  12123 452315  14013 663615 583531  96544 670105  39951
 288965 376549 219015 858200 264397   8197 226341   5847 569514 240148
  69362 167154  56745 228009  13814  19998 268259 300016 221406 214275]
=> pixel_per_image: [1151136  216972   27405  986769   31185 1849680 1665468  236439 1653057
   78876  650412  910476  638190 2529639  718956   18585  587727   13167
 1435959  660240  191331  436779  134505  496314   31626   41706  626535
  664587  741951  503181]
=> FF_term: [0.39386745 0.5025395  0.44236453 0.45837982 0.44935065 0.35877287
 0.35037059 0.40832519 0.4053732  0.50650388 0.44427993 0.41357378
 0.3431815  0.33925789 0.36775129 0.441054

In [104]:
over_metric = mMetric_v1(ori_activation_maps, opt_activation_maps, selected_filter=selected_filter)
print(over_metric)
print(np.mean(over_metric))

=> SF_0: [2817 1620   40 5115  111 1354  964  990 3058  724 3262 3116  378 1118
 1709    0 2487  100 1817 2145  993 1148  606 2447   69  294  488 4355
 1703 2795]
=> pixel_per_image: 50176
=> SF_term: [0.05614238 0.03228635 0.00079719 0.10194117 0.00221221 0.02698501
 0.01921237 0.01973055 0.06094547 0.01442921 0.06501116 0.0621014
 0.00753348 0.02228157 0.03406011 0.         0.04956553 0.00199298
 0.03621253 0.04274952 0.01979034 0.02287946 0.01207749 0.04876834
 0.00137516 0.00585938 0.00972577 0.08679448 0.03394053 0.05570392]
=> FF_0: [ 492798   97594   12228  427263   14617  794739  713764  104945  712065
   37374  283250  392341  273604 1084886  313137    8365  262264    5968
  615317  284171   83449  188036   58512  215085   14763   18907  268688
  286829  319588  220010]
=> pixel_per_image: [1151136  216972   27405  986769   31185 1849680 1665468  236439 1653057
   78876  650412  910476  638190 2529639  718956   18585  587727   13167
 1435959  660240  191331  436779  134505  49

##### mMetric_v2

In [82]:
over_metric = mMetric_v2(ori_activation_maps, ori_activation_maps, selected_filter=selected_filter)
print(over_metric)

[[1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 ...
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]]
=> SF_0: [1.9929847e-05 1.9929847e-05 1.9929847e-05 1.9929847e-05 1.9929847e-05
 1.9929847e-05 1.9929847e-05 1.9929847e-05 1.9929847e-05 1.9929847e-05
 1.9929847e-05 1.9929847e-05 1.9929847e-05 1.9929847e-05 1.9929847e-05
 1.9929847e-05 1.9929847e-05 1.9929847e-05 1.9929847e-05 1.9929847e-05
 1.9929847e-05 1.9929847e-05 1.9929847e-05 1.9929847e-05 1.9929847e-05
 1.9929847e-05 1.9929847e-05 1.9929847e-05 1.9929847e-05 1.9929847e-05]
=> pixel_per_image: 50176
=> SF_term: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1.]
ori_sel:  (30, 1, 224, 224)
ori_sel:  (30, 63, 224, 224)
=> FF_0: [453395 109037  12123 452315  14013 663615 583531  96544 670105  39951
 288965 376549 219015 858200 264397   8197 226341   5847 569514 240148
  69362 167154  56745 228009  13814  19998 268259 300016 221406 21427

In [83]:
over_metric = mMetric_v2(ori_activation_maps, opt_activation_maps, selected_filter=selected_filter)
print(over_metric)

[[0.50780404 0.5092825  0.5071508  ... 0.5260068  0.5130839  0.5076022 ]
 [0.43216452 0.42466876 0.42422515 ... 0.42543408 0.4303236  0.42432106]
 [0.16058587 0.16749926 0.16849303 ... 0.17126307 0.17901869 0.16740999]
 ...
 [0.3918673  0.41037104 0.40676403 ... 0.4118166  0.42114702 0.4018169 ]
 [0.46618858 0.48951557 0.49279281 ... 0.5152079  0.5159618  0.48156777]
 [0.23295479 0.23363301 0.2326551  ... 0.23018847 0.23404068 0.23428014]]
=> SF_0: [1.9929845e-05 1.9929850e-05 1.9929847e-05 1.9929850e-05 1.9929847e-05
 1.9929847e-05 1.9929850e-05 1.9929850e-05 1.9929847e-05 1.9929847e-05
 1.9929850e-05 1.9929847e-05 1.9929847e-05 1.9929847e-05 1.9929845e-05
 1.9929847e-05 1.9929850e-05 1.9929847e-05 1.9929847e-05 1.9929847e-05
 1.9929847e-05 1.9929847e-05 1.9929847e-05 1.9929845e-05 1.9929847e-05
 1.9929847e-05 1.9929847e-05 1.9929845e-05 1.9929850e-05 1.9929850e-05]
=> pixel_per_image: 50176
=> SF_term: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1.

##### mMetric_v3

In [10]:
#over_metric = 
mMetric_v3(ori_activation_maps, ori_activation_maps, selected_filter=selected_filter)
#print(over_metric)
#print(np.mean(over_metric))

tensor(1., device='cuda:0')
tensor(1., device='cuda:0')
tensor(1., device='cuda:0')
tensor(1., device='cuda:0')
tensor(1., device='cuda:0')
tensor(1., device='cuda:0')
tensor(1., device='cuda:0')
tensor(1., device='cuda:0')
tensor(1., device='cuda:0')
tensor(1., device='cuda:0')
tensor(1., device='cuda:0')
tensor(1., device='cuda:0')
tensor(1., device='cuda:0')
tensor(1., device='cuda:0')
tensor(1., device='cuda:0')
tensor(1., device='cuda:0')
tensor(1., device='cuda:0')
tensor(1., device='cuda:0')
tensor(1., device='cuda:0')
tensor(1., device='cuda:0')
tensor(1., device='cuda:0')
tensor(1., device='cuda:0')
tensor(1., device='cuda:0')
tensor(1., device='cuda:0')
tensor(1., device='cuda:0')
tensor(1., device='cuda:0')
tensor(1., device='cuda:0')
tensor(1., device='cuda:0')
tensor(1., device='cuda:0')
tensor(1., device='cuda:0')
ori_sel:  (30, 1, 224, 224)
ori_sel:  (30, 63, 224, 224)
torch.Size([30, 63, 224, 224])
tensor(0.7475, device='cuda:0')


In [11]:
mMetric_v3(ori_activation_maps, opt_activation_maps, selected_filter=selected_filter)

tensor(0.8901, device='cuda:0')
tensor(0.9933, device='cuda:0')
tensor(0.8353, device='cuda:0')
tensor(0.9529, device='cuda:0')
tensor(0.9891, device='cuda:0')
tensor(0.8176, device='cuda:0')
tensor(0.9062, device='cuda:0')
tensor(0.8576, device='cuda:0')
tensor(0.8890, device='cuda:0')
tensor(0.8627, device='cuda:0')
tensor(0.9783, device='cuda:0')
tensor(0.9218, device='cuda:0')
tensor(0.9436, device='cuda:0')
tensor(0.8735, device='cuda:0')
tensor(0.9659, device='cuda:0')
tensor(0.9956, device='cuda:0')
tensor(0.9568, device='cuda:0')
tensor(0.7212, device='cuda:0')
tensor(0.9255, device='cuda:0')
tensor(0.8758, device='cuda:0')
tensor(0.8081, device='cuda:0')
tensor(0.9399, device='cuda:0')
tensor(0.9892, device='cuda:0')
tensor(0.8774, device='cuda:0')
tensor(0.8973, device='cuda:0')
tensor(0.7582, device='cuda:0')
tensor(0.8007, device='cuda:0')
tensor(0.9138, device='cuda:0')
tensor(0.9369, device='cuda:0')
tensor(0.9477, device='cuda:0')
ori_sel:  (30, 1, 224, 224)
ori_sel:  (3

### L3F28

In [150]:
exp = "7004"
selected_layer = 3
selected_filter = 28
class_index = 950
ori_activation_maps, opt_activation_maps = main(exp, selected_layer, 
                                                selected_filter, class_index)

=> class index: 950
Only Keep 3th layers before.
Resume from model from exp: 037 at epoch 0
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 224, 224]           1,792
              ReLU-2         [-1, 64, 224, 224]               0
            Conv2d-3         [-1, 64, 224, 224]          36,928
              ReLU-4         [-1, 64, 224, 224]               0
             VGG16-5         [-1, 64, 224, 224]               0
Total params: 38,720
Trainable params: 38,720
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.57
Forward/backward pass size (MB): 122.50
Params size (MB): 0.15
Estimated Total Size (MB): 123.22
----------------------------------------------------------------
Len of new dataset is :30
../../../saved/pack/7004
Load from ../../../saved/pack/7004/127203/n07747607_10061.png
Load from ../../../saved/pa

#### mMetric_v1

In [108]:
over_metric = mMetric_v1(ori_activation_maps, ori_activation_maps, selected_filter=selected_filter)
print(over_metric)
print(np.mean(over_metric))

=> SF_0: [50176 50176 50176 50176 50176 50176 50176 50176 50176 50176 50176 50176
 50176 50176 50176 50176 50176 50176 50176 50176 50176 50176 50176 50176
 50176 50176 50176 50176 50176 50176]
=> pixel_per_image: 50176
=> SF_term: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1.]
=> FF_0: [ 912666  773575  264145  608296  726332  941569  982697 1056030  680586
   48262 1513034 1197917  519385 1253848  351201   29712 1491286  327659
  291539  234548  556699  322950  827831  180334   53659   46614   13855
 1079197  528770  436783]
=> pixel_per_image: [1649403 1440684  526302 1130346 1348137 1804824 1628109 1970514 1270962
   91728 2681658 2119824  911673 2279214  600516   64386 2685312  624582
  551439  452466 1050147  608265 1541988  342153  107730   95634   24003
 1987335 1047249  779562]
=> FF_term: [0.55333111 0.53694981 0.50188865 0.53815027 0.5387672  0.52169574
 0.60358182 0.53591601 0.53548887 0.52614251 0.56421587 0.5651021
 0.56970537 

In [109]:
over_metric = mMetric_v1(ori_activation_maps, opt_activation_maps, selected_filter=selected_filter)
print(over_metric)
print(np.mean(over_metric))

=> SF_0: [22570 19390  8171 15749 21207 28043 23635 29391 16610   894 39868 24687
 12393 34205  7242   977 40877  9291  7001  6808 15413  8561 22206  4226
  1478  1425   255 25388 16381 10570]
=> pixel_per_image: 50176
=> SF_term: [0.44981665 0.38643973 0.16284678 0.31387516 0.42265226 0.5588927
 0.47104193 0.58575813 0.33103476 0.01781728 0.79456314 0.49200813
 0.24699059 0.68170041 0.14433195 0.01947146 0.81467235 0.18516821
 0.13952886 0.1356824  0.30717873 0.17061942 0.44256218 0.08422353
 0.02945631 0.02840003 0.00508211 0.50597895 0.32647082 0.21065848]
=> FF_0: [ 906805  784931  288335  621890  754363 1071642  978132 1092775  724176
   50636 1488188 1240115  537844 1344127  356549   34924 1505613  346063
  309068  267156  581099  343917  865726  189180   59016   52588   13567
 1073242  601936  451736]
=> pixel_per_image: [1649403 1440684  526302 1130346 1348137 1804824 1628109 1970514 1270962
   91728 2681658 2119824  911673 2279214  600516   64386 2685312  624582
  551439  4524

In [66]:
mMetric_v1(ori_activation_maps, ori_activation_maps, selected_filter=selected_filter)

=> SF_0: [26181 22868  8354 17942 21399 28648 25843 31278 20174  1456 42566 33648
 14471 36178  9532  1022 42624  9914  8753  7182 16669  9655 24476  5431
  1710  1518   381 31545 16623 12374]
=> pixel_per_image: 3211264
=> SF_term: [0.00815286 0.00712118 0.00260147 0.00558721 0.00666373 0.0089211
 0.00804761 0.00974009 0.00628226 0.0004534  0.01325522 0.01047812
 0.00450633 0.01126597 0.0029683  0.00031825 0.01327328 0.00308726
 0.00272572 0.0022365  0.00519079 0.0030066  0.00762192 0.00169123
 0.0005325  0.00047271 0.00011864 0.00982323 0.00517647 0.00385331]
ori_sel:  (30, 1, 224, 224)
ori_sel:  (30, 63, 224, 224)
=> FF_0: [ 912666  773575  264145  608296  726332  941569  982697 1056030  680586
   48262 1513034 1197917  519385 1253848  351201   29712 1491286  327659
  291539  234548  556699  322950  827831  180334   53659   46614   13855
 1079197  528770  436783]
=> pixel_per_image: [1649403 1440684  526302 1130346 1348137 1804824 1628109 1970514 1270962
   91728 2681658 2119824  91

array([0.28074199, 0.2720355 , 0.25224506, 0.27186874, 0.27271547,
       0.26530842, 0.30581472, 0.27282805, 0.27088556, 0.26329796,
       0.28873554, 0.28779011, 0.28710585, 0.28069447, 0.29390017,
       0.23089252, 0.28431126, 0.26384626, 0.26570669, 0.26030679,
       0.26765305, 0.26697148, 0.27224074, 0.26437392, 0.24931016,
       0.24394675, 0.28866908, 0.27643026, 0.25504491, 0.28207307])

#### mMetric_v3

In [151]:
mMetric_v3(ori_activation_maps, ori_activation_maps, selected_filter=selected_filter)

tensor(1., device='cuda:0')
tensor(1., device='cuda:0')
tensor(1., device='cuda:0')
tensor(1., device='cuda:0')
tensor(1., device='cuda:0')
tensor(1., device='cuda:0')
tensor(1., device='cuda:0')
tensor(1., device='cuda:0')
tensor(1., device='cuda:0')
tensor(1., device='cuda:0')
tensor(1., device='cuda:0')
tensor(1., device='cuda:0')
tensor(1., device='cuda:0')
tensor(1., device='cuda:0')
tensor(1., device='cuda:0')
tensor(1., device='cuda:0')
tensor(1., device='cuda:0')
tensor(1., device='cuda:0')
tensor(1., device='cuda:0')
tensor(1., device='cuda:0')
tensor(1., device='cuda:0')
tensor(1., device='cuda:0')
tensor(1., device='cuda:0')
tensor(1., device='cuda:0')
tensor(1., device='cuda:0')
tensor(1., device='cuda:0')
tensor(1., device='cuda:0')
tensor(1., device='cuda:0')
tensor(1., device='cuda:0')
tensor(1., device='cuda:0')


In [152]:
mMetric_v3(ori_activation_maps, opt_activation_maps, selected_filter=selected_filter)

tensor(0.8420, device='cuda:0')
tensor(0.9733, device='cuda:0')
tensor(0.9766, device='cuda:0')
tensor(0.9778, device='cuda:0')
tensor(0.9523, device='cuda:0')
tensor(0.9026, device='cuda:0')
tensor(0.9297, device='cuda:0')
tensor(0.9357, device='cuda:0')
tensor(0.9129, device='cuda:0')
tensor(0.9992, device='cuda:0')
tensor(0.8277, device='cuda:0')
tensor(0.9168, device='cuda:0')
tensor(0.9587, device='cuda:0')
tensor(0.9415, device='cuda:0')
tensor(0.9707, device='cuda:0')
tensor(0.9918, device='cuda:0')
tensor(0.8260, device='cuda:0')
tensor(0.9729, device='cuda:0')
tensor(0.9726, device='cuda:0')
tensor(0.9675, device='cuda:0')
tensor(0.9577, device='cuda:0')
tensor(0.9344, device='cuda:0')
tensor(0.9131, device='cuda:0')
tensor(0.9921, device='cuda:0')
tensor(0.9881, device='cuda:0')
tensor(0.9869, device='cuda:0')
tensor(0.9883, device='cuda:0')
tensor(0.9493, device='cuda:0')
tensor(0.9405, device='cuda:0')
tensor(0.9496, device='cuda:0')


In [148]:
exp = "70071"
selected_layer = 11
selected_filter = 75
class_index = 950
ori_activation_maps, opt_activation_maps = main(exp, selected_layer, 
                                                selected_filter, class_index)
#over_metric = mMetric_v1(ori_activation_maps, opt_activation_maps, selected_filter=selected_filter)
over_metric = mMetric_v2(ori_activation_maps, opt_activation_maps, selected_filter=selected_filter)
print(over_metric)

=> class index: 950
Only Keep 11th layers before.
Resume from model from exp: 037 at epoch 0
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 224, 224]           1,792
              ReLU-2         [-1, 64, 224, 224]               0
            Conv2d-3         [-1, 64, 224, 224]          36,928
              ReLU-4         [-1, 64, 224, 224]               0
         MaxPool2d-5         [-1, 64, 112, 112]               0
            Conv2d-6        [-1, 128, 112, 112]          73,856
              ReLU-7        [-1, 128, 112, 112]               0
            Conv2d-8        [-1, 128, 112, 112]         147,584
              ReLU-9        [-1, 128, 112, 112]               0
        MaxPool2d-10          [-1, 128, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]         295,168
             ReLU-12          [-1, 256, 56, 56]               0
          

## Main

In [5]:
# excel_path = "Single.xlsx"
excel_path = "../week9/Batches.xlsx"
sheet = "950"
# sheet = "950"
excepts = ["052148"]

In [7]:
xlsx = pd.ExcelFile(excel_path)
excel = pd.read_excel(xlsx, sheet, Sdtype={"exp": str, 
                                           "selected_layer": int,
                                           "selected_filter": int, 
                                           "class_index": int,
                                            "Done": str}).values
for data in excel:
    exp, selected_layer, selected_filter, class_index, done = data[0], data[1], \
        data[2], data[3], data[4]
    exp = exp.replace("x", "")
    if exp in excepts:
        print("=> Skip")
    else:
        if done == "N":
            print("=> RUN ", exp, selected_layer, selected_filter, class_index)
            # main(exp, selected_layer, selected_filter, class_index)
            ori_activation_maps, opt_activation_maps = main(exp, 
                                                            selected_layer, 
                                                            selected_filter, 
                                                            class_index)
            mMetric_v3(ori_activation_maps, opt_activation_maps, 
                       selected_filter=selected_filter) 
            print("")
            print("-"*50)

=> RUN  061300 1 16 950
=> class index: 950
Only Keep 1th layers before.
Resume from model from exp: 037 at epoch 0
Len of new dataset is :30
../../../saved/pack/061300
=> Register fhook 1
=> Register fhook 1
RMSE: 0.17237147688865662 - std 0.17498500645160675
SSIM_SEL: 0.9007095098495483 - std 0.06958510726690292
SSIM_RES: 0.8071271777153015 - std 0.13045555353164673
SSIM_SUM 0.8539183139801025 - std 0.06818907707929611

--------------------------------------------------
=> RUN  052490 1 16 950
=> class index: 950
Only Keep 1th layers before.
Resume from model from exp: 037 at epoch 0
Len of new dataset is :30
../../../saved/pack/052490
=> Register fhook 1
=> Register fhook 1
RMSE: 0.17237147688865662 - std 0.17498500645160675
SSIM_SEL: 0.9007095098495483 - std 0.06958510726690292
SSIM_RES: 0.8071271777153015 - std 0.13045555353164673
SSIM_SUM 0.8539183139801025 - std 0.06818907707929611

--------------------------------------------------
=> RUN  7001 1 47 950
=> class index: 950
Only

### Class 14

In [15]:
# excel_path = "Single.xlsx"
excel_path = "../week9/Batches.xlsx"
sheet = "Others"
# sheet = "950"
excepts = ["052148"]

In [ ]:
xlsx = pd.ExcelFile(excel_path)
excel = pd.read_excel(xlsx, sheet, Sdtype={"exp": str, 
                                           "selected_layer": int,
                                           "selected_filter": int, 
                                           "class_index": int,
                                            "Done": str}).values
for data in excel:
    exp, selected_layer, selected_filter, class_index, done = data[0], data[1], \
        data[2], data[3], data[4]
    exp = exp.replace("x", "")
    if exp in excepts:
        print("=> Skip")
    else:
        if done == "N":
            print("=> RUN ", exp, selected_layer, selected_filter, class_index)
            # main(exp, selected_layer, selected_filter, class_index)
            ori_activation_maps, opt_activation_maps = main(exp, 
                                                            selected_layer, 
                                                            selected_filter, 
                                                            class_index)
            mMetric_v3(ori_activation_maps, opt_activation_maps, 
                       selected_filter=selected_filter) 
            print("")
            print("-"*50)

=> RUN  052145 6 19 968
=> class index: 968
Only Keep 6th layers before.
Resume from model from exp: 037 at epoch 0
Len of new dataset is :30
../../../saved/pack/052145
=> Register fhook 6
=> Register fhook 6
RMSE: 1.402826189994812 - std 0.5411962270736694
SSIM_SEL: 0.1796797215938568 - std 0.08075942099094391
SSIM_RES: 0.3676646649837494 - std 0.16478444635868073
SSIM_SUM 0.2736721932888031 - std 0.08494801819324493

--------------------------------------------------
=> RUN  052146 6 19 878
=> class index: 878
Only Keep 6th layers before.
Resume from model from exp: 037 at epoch 0
Len of new dataset is :30
../../../saved/pack/052146
=> Register fhook 6
=> Register fhook 6
RMSE: 1.7570254802703857 - std 0.46729645133018494
SSIM_SEL: 0.14526595175266266 - std 0.06616485118865967
SSIM_RES: 0.27733907103538513 - std 0.08015528321266174
SSIM_SUM 0.21130253374576569 - std 0.04024346172809601

--------------------------------------------------
=> RUN  052147 6 19 90
=> class index: 90
Only 

Len of new dataset is :30
../../../saved/pack/052102
=> Register fhook 29
=> Register fhook 29
RMSE: 0.5485287308692932 - std 0.22176256775856018
SSIM_SEL: 0.8533899188041687 - std 0.08572357892990112
SSIM_RES: 0.8930657505989075 - std 0.023727698251605034
SSIM_SUM 0.8732278347015381 - std 0.04478314518928528

--------------------------------------------------
=> RUN  052103 29 334 14
=> class index: 14
Only Keep 29th layers before.
Resume from model from exp: 037 at epoch 0
Len of new dataset is :30
../../../saved/pack/052103
=> Register fhook 29
=> Register fhook 29
RMSE: 0.2540377676486969 - std 0.16776616871356964
SSIM_SEL: 0.8214017748832703 - std 0.1284390687942505
SSIM_RES: 0.9043479561805725 - std 0.021509870886802673
SSIM_SUM 0.8628748655319214 - std 0.06285174936056137

--------------------------------------------------
=> RUN  052105 27 161 968
=> class index: 968
Only Keep 27th layers before.
Resume from model from exp: 037 at epoch 0
Len of new dataset is :30
../../../save